# Sentiment-Based Trading Model Training

This notebook trains a neural network model that incorporates news sentiment as a feature for predicting stock price movements.

## Data collection

In [140]:
import os
import sys
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from pathlib import Path
from tqdm.notebook import tqdm
import pickle
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest, StockLatestQuoteRequest
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import time

from trading_bot_llm_sentiment_brian import TradingBotLLMSentiment

In [141]:
load_dotenv()

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Initialize the trading bot to use its data collection methods
bot = TradingBotLLMSentiment()
print(f"Bot initialized with symbols: {bot.symbols}")

# Create data directory if it doesn't exist
os.makedirs('data', exist_ok=True)

2025-03-29 07:44:52,177 - trading_bot_llm_sentiment_brian - INFO - Trading bot trading_bot_llm_sentiment_brian initialized with symbols: ['AAPL', 'MSFT', 'META', 'GOOGL', 'AMZN', 'NVDA']
Bot initialized with symbols: ['AAPL', 'MSFT', 'META', 'GOOGL', 'AMZN', 'NVDA']


In [142]:
days=365
symbols = ['AAPL', 'MSFT', 'META', 'GOOGL', 'AMZN', 'NVDA']

symbol = 'AAPL'
filename = 'combined_historical_with_daily_sentiment.csv'

In [143]:
api_key = os.environ['ALPACA_API_KEY']
api_secret = os.environ['ALPACA_API_SECRET']
data_client = StockHistoricalDataClient(api_key, api_secret)

In [31]:
df = bot.get_historical_data(symbol, days=days)

2025-03-27 21:29:53,934 - trading_bot_llm_sentiment_brian - INFO - Retrieved 250 bars for AAPL


In [48]:
df.head()

,symbol,timestamp,open,high,low,close,volume,trade_count,vwap
0,AAPL,2025-03-17 04:00:00+00:00,213.31,215.2200,209.97,214.00,48073426.0,577436.0,213.202242
1,AAPL,2025-03-18 04:00:00+00:00,214.16,215.1500,211.49,212.69,42432426.0,493004.0,213.109470
2,AAPL,2025-03-19 04:00:00+00:00,214.22,218.7600,213.75,215.24,54385391.0,524678.0,215.609629
3,AAPL,2025-03-20 04:00:00+00:00,213.99,217.4899,212.22,214.10,48862947.0,499769.0,214.396693
4,AAPL,2025-03-21 04:00:00+00:00,211.56,218.8400,211.28,218.27,94127768.0,496948.0,215.734078


In [5]:
def collect_historical_data_with_daily_sentiment(symbol, days=365):
    """
    Collect historical price data and daily sentiment data for a given symbol.
    
    Args:
        symbol (str): Stock symbol.
        days (int): Number of days of historical data to collect.
        
    Returns:
        DataFrame: Combined price and sentiment data.
    """
    print(f"Collecting data for {symbol}...")
    
    # Get historical price data
    df = bot.get_historical_data(symbol, days=days)
    if df is None:
        print(f"No historical data found for {symbol}")
        return None
    
    # Create a copy to avoid modifying the original dataframe
    df = df.copy()
    
    # Extract the date from the timestamp for daily grouping
    df['date'] = pd.to_datetime(df['timestamp']).dt.date
    daily_dates = df['date'].unique().tolist()
    
    print(f"Collected {len(df)} price data points, calculating sentiment for {len(daily_dates)} days...")
    
    # Add sentiment column
    df['sentiment'] = np.nan
    
    # Get sentiment for each day
    for date in daily_dates:
        date_str = pd.to_datetime(date).strftime('%Y-%m-%d')
        print(f"Getting sentiment for {symbol} for {date_str}")
        
        articles = 10
        news_date = pd.to_datetime(date)
        # Using a 1-day lookback range to fetch daily sentiment
        lookback_range = timedelta(days=1)
        
        sentiment = bot.get_sentiment_signal(symbol, articles, news_date, lookback_range)
        df.loc[df['date'] == date, 'sentiment'] = sentiment
        time.sleep(1)
    
    
    # Drop rows with NaN values
    # df = df.dropna()
    
    print(f"Final dataset: {len(df)} rows for {symbol}")
    return df


## Process all symbols and save the dataset

In [ ]:
# List to store dataframes for each symbol
dfs = []

for symbol in symbols:
    try:
        data = collect_historical_data_with_daily_sentiment(symbol)
        if data is not None:
            # Optionally add a symbol column if you want a combined DF later
            data['symbol'] = symbol  
            
            # Save individual CSV for each symbol
            data.to_csv(f"data/{symbol}_historical_with_daily_sentiment.csv", index=False)
            print(f"Saved data for {symbol}")
            
            # Append to our list for later combining
            dfs.append(data)
        else:
            print(f"No data found for {symbol}")
    except Exception as e:
        print(f"Error processing {symbol}: {e}")

# If you want a single combined DataFrame for all symbols:
if dfs:
    combined_df = pd.concat(dfs, ignore_index=True)
    combined_df.to_csv("data/combined_historical_with_daily_sentiment.csv", index=False)
    print("Saved combined data for all symbols.")
else:
    print("No data to combine.")


## Model Training

In [168]:
SEQ_LENGTH = 30  # use past 30 days
FEATURES = ['open', 'high', 'low', 'close', 'volume', 'sentiment'] 
TRAIN_RATIO = 0.8

# Identify which features to scale (excluding sentiment which is already -1 to 1)
FEATURES_TO_SCALE = ['open', 'high', 'low', 'close', 'volume']


In [169]:
combined_file = "data/combined_historical_with_daily_sentiment.csv"
if not os.path.exists(combined_file):
    raise FileNotFoundError(f"{combined_file} does not exist.")


In [170]:
df = pd.read_csv(combined_file, index_col=0)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values(['symbol', 'timestamp'])

In [171]:
df.tail(1)

,timestamp,open,high,low,close,volume,trade_count,vwap,date,sentiment
symbol,,,,,,,,,,
NVDA,2025-03-28 04:00:00+00:00,111.485,112.87,109.0701,109.67,229872549.0,1847538.0,110.119953,2025-03-28,0.7


In [177]:
df.index

Index(['AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL', 'AAPL',
       'AAPL',
       ...
       'NVDA', 'NVDA', 'NVDA', 'NVDA', 'NVDA', 'NVDA', 'NVDA', 'NVDA', 'NVDA',
       'NVDA'],
      dtype='object', name='symbol', length=1512)

In [172]:
scaler = MinMaxScaler()
df[FEATURES_TO_SCALE] = scaler.fit_transform(df[FEATURES_TO_SCALE])


In [173]:
with open('data/scaler.pkl', 'wb') as f:
    pickle.dump({
        'scaler': scaler,
        'features_to_scale': FEATURES_TO_SCALE
    }, f)


In [176]:
df.head()

,timestamp,open,high,low,close,volume,trade_count,vwap,date,sentiment
symbol,,,,,,,,,,
AAPL,2024-03-27 04:00:00+00:00,0.068224,0.060904,0.072695,0.066105,0.068231,670630.0,172.581734,2024-03-27,0.0
AAPL,2024-03-28 04:00:00+00:00,0.069391,0.059716,0.073061,0.064479,0.074863,648027.0,171.355595,2024-03-28,0.0
AAPL,2024-04-01 04:00:00+00:00,0.068903,0.058865,0.072114,0.063190,0.050994,676830.0,170.066484,2024-04-01,-0.7
AAPL,2024-04-02 04:00:00+00:00,0.067066,0.057208,0.070974,0.062133,0.054821,608917.0,168.895583,2024-04-02,-0.5
AAPL,2024-04-03 04:00:00+00:00,0.066814,0.058371,0.071295,0.062853,0.052777,571286.0,169.864966,2024-04-03,-0.3


In [179]:
df.reset_index(inplace=True)
symbol_dummies = pd.get_dummies(df['symbol'], prefix='symbol').astype('float32')
df = pd.concat([df, symbol_dummies], axis=1)

In [180]:
symbol_columns = symbol_dummies.columns.tolist()
all_features = FEATURES + symbol_columns

In [181]:
print(all_features)

['open', 'high', 'low', 'close', 'volume', 'sentiment', 'symbol_AAPL', 'symbol_AMZN', 'symbol_GOOGL', 'symbol_META', 'symbol_MSFT', 'symbol_NVDA']


In [182]:
X_train_list, y_train_list = [], []
X_test_list, y_test_list = [], []

In [183]:
def create_sequences(df, seq_length=SEQ_LENGTH, feature_columns=all_features):
    """
    Create sequences from the DataFrame using a sliding window.
    For each sequence of past `seq_length` days, the target is the closing price on day seq_length+1.
    """
    X, y = [], []
    if len(df) < seq_length + 1:
        return None, None
    for i in range(len(df) - seq_length):
        # Sequence of features for past seq_length days
        seq = df.iloc[i:i+seq_length][feature_columns].values
        # Target is next day's closing price
        target = df.iloc[i+seq_length]['close']
        X.append(seq)
        y.append(target)
    return np.array(X), np.array(y)

In [184]:
for symbol, group in df.groupby('symbol'):
    group = group.sort_values('timestamp').reset_index(drop=True)
    
    X_symbol, y_symbol = create_sequences(group, seq_length=SEQ_LENGTH)
    
    if X_symbol is None or len(X_symbol) == 0:
        print(f"Not enough data for {symbol}; skipping.")
        continue
    
    # Split data by time (first TRAIN_RATIO for training, rest for testing)
    split_idx = int(len(X_symbol) * TRAIN_RATIO)
    X_train_list.append(X_symbol[:split_idx])
    y_train_list.append(y_symbol[:split_idx])
    X_test_list.append(X_symbol[split_idx:])
    y_test_list.append(y_symbol[split_idx:])
    
    print(f"{symbol}: {len(X_symbol)} sequences, {split_idx} training samples and {len(X_symbol) - split_idx} testing samples.")



AAPL: 222 sequences, 177 training samples and 45 testing samples.
AMZN: 222 sequences, 177 training samples and 45 testing samples.
GOOGL: 222 sequences, 177 training samples and 45 testing samples.
META: 222 sequences, 177 training samples and 45 testing samples.
MSFT: 222 sequences, 177 training samples and 45 testing samples.
NVDA: 222 sequences, 177 training samples and 45 testing samples.


In [185]:
X_train = np.concatenate(X_train_list, axis=0)
y_train = np.concatenate(y_train_list, axis=0)
X_test = np.concatenate(X_test_list, axis=0)
y_test = np.concatenate(y_test_list, axis=0)

In [186]:
print("Combined training shape:", X_train.shape)
print("Combined testing shape:", X_test.shape)

Combined training shape: (1062, 30, 12)
Combined testing shape: (270, 30, 12)


In [159]:
model = Sequential()
model.add(LSTM(128, activation='relu', return_sequences=True, input_shape=(SEQ_LENGTH, num_features)))
model.add(Dropout(0.2))  # Dropout to reduce overfitting

# Second LSTM layer
model.add(LSTM(64, activation='relu'))
model.add(Dropout(0.2))

# An intermediate dense layer
model.add(Dense(16, activation='relu'))

# Final output layer for regression
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')
model.summary()

/Users/I523193/.local/pipx/venvs/jupyter/lib/python3.12/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_8 (LSTM)                   │ (None, 30, 128)        │        72,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 30, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_9 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 16)             │         1,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 122,657 (479.13 KB)

 Trainable params: 122,657 (479.13 KB)

 Non-trainable params: 0 (0.00 B)

In [187]:
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

In [188]:
history = model.fit(X_train, y_train, 
                    epochs=100, batch_size=32, 
                    validation_split=0.2, 
                    callbacks=[reduce_lr])

Epoch 1/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 23ms/step - loss: 37130.2070 - val_loss: 116.6497 - learning_rate: 1.0000e-04
Epoch 2/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 29.7888 - val_loss: 45.8966 - learning_rate: 1.0000e-04
Epoch 3/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - loss: 17.1469 - val_loss: 41.3592 - learning_rate: 1.0000e-04
Epoch 4/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 14.4220 - val_loss: 37.3454 - learning_rate: 1.0000e-04
Epoch 5/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step - loss: 8.8938 - val_loss: 16.9801 - learning_rate: 1.0000e-04
Epoch 6/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 3.9328 - val_loss: 5.8792 - learning_rate: 1.0000e-04
Epoch 7/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 2.1208 - val_loss: 2.8102 - learning_rate: 1.0000e-04
Epoch 8/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 1.3311 - val_loss: 1.7059 - learning_rate: 1.0000e-04
Epoch 9/100
27/27 ━━━━━━━━━━━━━━━━━━━━ 1s 21ms/step - loss: 0.9499 - val_lo

In [189]:
test_loss = model.evaluate(X_test_scaled, y_test)
print("Test Loss (MSE):", test_loss)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.0015
Test Loss (MSE): 0.0027952813543379307


In [190]:
now = datetime.now().strftime('%Y-%m-%d')
print(now)

2025-03-29


In [191]:
os.makedirs('data/models', exist_ok=True)
date = datetime.now().strftime('%Y-%m-%d')
path = 'data'
model_name = f"lstm_combined_model_{date}.keras"
model.save(f"data/models/{model_name}")
print(f"Saved LSTM model to models/{model_name}")

Saved LSTM model to models/lstm_combined_model_2025-03-29.keras


In [203]:
def predict_todays_closing_price_enriched(symbol):
    """
    Predict today's closing price for the given symbol using enriched price data that includes sentiment.
    
    Process:
    1. Load and update historical sentiment data.
    2. Scale the price and volume features (same as during training).
    3. Add one-hot encoding for symbols.
    4. Create the input sequence for the model.
    5. Predict the closing price.
    
    Returns:
        float or None: The predicted closing price for today, or None if not enough data.
    """
    # Load the scaler
    with open('data/scaler.pkl', 'rb') as f:
        scaler_info = pickle.load(f)
        SCALER = scaler_info['scaler']
        FEATURES_TO_SCALE = scaler_info['features_to_scale']
    
    # Load the trained model
    MODEL = load_model(f"data/models/lstm_combined_model_2025-03-29.keras")
    
    # Step 1: Load and update historical sentiment data
    df = bot.load_and_update_sentiment_data(SEQ_LENGTH)
    if df is None or df.empty:
        print("Failed to load sentiment data.")
        return None
    
    # Step 2: Scale numerical features using the same scaler from training
    df_scaled = df.copy()
    df_scaled[FEATURES_TO_SCALE] = SCALER.transform(df[FEATURES_TO_SCALE])
    
    # Step 3: Add one-hot encoding for symbols
    symbol_dummies = pd.get_dummies(df_scaled['symbol'], prefix='symbol').astype('float32')
    df_scaled = pd.concat([df_scaled, symbol_dummies], axis=1)
    
    # Get the full list of features for the model
    symbol_columns = [col for col in df_scaled.columns if col.startswith('symbol_')]
    all_features = FEATURES + symbol_columns
    
    # Step 4: Filter for the specific symbol and check data sufficiency
    symbol_df = df_scaled[df_scaled['symbol'] == symbol].sort_values(by="timestamp")
    if len(symbol_df) < SEQ_LENGTH:
        print(f"Not enough data for {symbol}. Need {SEQ_LENGTH} days, have {len(symbol_df)}.")
        return None
    
    # Create input sequence using the last SEQ_LENGTH rows
    input_seq = symbol_df.iloc[-SEQ_LENGTH:][all_features].values.astype('float32')
    input_seq = input_seq.reshape(1, SEQ_LENGTH, len(all_features))
    
    # Step 5: Make the prediction (scaled)
    predicted_scaled = MODEL.predict(input_seq, verbose=0)[0][0]
    
    # Step 6: Unscale the prediction
    # Get the index of 'close' in FEATURES_TO_SCALE
    close_idx = FEATURES_TO_SCALE.index('close')
    
    # Create a dummy array with zeros for all scalable features
    dummy = np.zeros((1, len(FEATURES_TO_SCALE)))
    # Place the scaled prediction in the position corresponding to 'close'
    dummy[0, close_idx] = predicted_scaled
    
    # Inverse transform to get the actual price
    unscaled_dummy = SCALER.inverse_transform(dummy)
    # Extract the unscaled closing price
    predicted_price = unscaled_dummy[0, close_idx]    
    print(f"Predicted closing price for {symbol} is ${predicted_price:.2f}")
    
    return predicted_price

In [208]:
symbol = 'NVDA'

In [209]:
prediction = predict_todays_closing_price_enriched(symbol)

2025-03-29 16:21:58,181 - trading_bot_llm_sentiment_brian - INFO - Loaded combined historical data from data/combined_historical_with_daily_sentiment.csv
2025-03-29 16:21:58,275 - trading_bot_llm_sentiment_brian - INFO - Retrieved 21 bars for AAPL
2025-03-29 16:21:58,373 - trading_bot_llm_sentiment_brian - INFO - Retrieved 21 bars for MSFT
2025-03-29 16:21:58,480 - trading_bot_llm_sentiment_brian - INFO - Retrieved 21 bars for META
2025-03-29 16:21:58,592 - trading_bot_llm_sentiment_brian - INFO - Retrieved 21 bars for GOOGL
2025-03-29 16:21:58,694 - trading_bot_llm_sentiment_brian - INFO - Retrieved 21 bars for AMZN
2025-03-29 16:21:58,804 - trading_bot_llm_sentiment_brian - INFO - Retrieved 21 bars for NVDA
2025-03-29 16:21:58,808 - trading_bot_llm_sentiment_brian - INFO - Combined historical data is up-to-date.
Predicted closing price for NVDA is $209.35


In [8]:
len(df)

180